# Data catalog export

https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_get_tableGenerated with Gemini and tested on Jun 18 2024

In [ ]:
from google.cloud import bigquery

In [ ]:
project_id = ""
dataset_id = ""

client = bigquery.Client(project=project_id)

dataset_ref = client.dataset(dataset_id)
tables = client.list_tables(dataset_ref)
tables

In [ ]:

table_info = []
for table in tables:
    table_ref = dataset_ref.table(table.table_id)
    table_obj = client.get_table(table_ref)  # Get table details
    
    print(table_obj)
    
    # Extract Schema
    schema_info = []
    for field in table_obj.schema:
        schema_info.append({
            'name': field.name,
            'type': field.field_type,
            'mode': field.mode,
            'policy_tags': field.policy_tags.names if field.policy_tags else None
        })

    # Store Table Information
    table_info.append({
        'table_id': table.table_id,
        'schema': schema_info,
    })
    
    print(table_info)